In [1]:
# Run me to hide code cells
from IPython.display import display, HTML
# from IPython.core.display import display, HTML
display(HTML(r"""<style id=hide>div.input{display:none;}</style><button type="button"onclick="var myStyle = 
document.getElementById('hide').sheet;myStyle.insertRule('div.input{display:inherit !important;}', 0);">Show inputs</button>"""))

In [2]:
import os
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import set_matplotlib_formats
from glob import glob
import random
from IPython.display import display
from ipywidgets import interact
import ipywidgets as widgets
import skimage
import pathlib
import panel as pn
import pandas as pd 
pn.extension('tabulator')

import src.ImagePreprocessFilters as IPrep
import src.ImageParser as IP
import src.jupyter_functions as JN
from src.file_specs import FileSpecifics

def set_params(res=300):
    plt.rcParams['figure.dpi'] = res
    plt.rcParams['savefig.dpi'] = res

cmap = plt.cm.gray   # for black and white
set_params(res=300)


FOR IMAGES THAT ARE A STACK OF TIFFs
Each image is a stack of channels 

Folder structure:

-- Folder

    -- Stack_Pat1
    
    -- Stack_Pat2
    
    -- Stack_PatX
    
The channel is an index (from 0 to N channels) 


In [3]:
# # defining the Jupyter Widget 
def widget(files,channel_names):
    # helping functions
    def plot_percentile(images, COL_PLOT=2, RES=300):
        set_params(res=RES)
        if COL_PLOT>1:
            JN.plot_one_channel_side_by_side(images, columns=COL_PLOT, figsize=(50,50), cmap = cmap)
        else:
            JN.plot_one_channel(images, cmap = cmap) 

    def plot_comparison(images_or, imgs_filtered, res_compare, high_contrast):
        set_params(res=res_compare)
        JN.plot_compare_images(images_or, imgs_filtered, cmap = cmap, high_contrast = high_contrast)
    
    def plot_comparison_zoom(images_or, imgs_filtered, res_compare, high_contrast):
        output.clear_output()
        set_params(res=res_compare)
        JN.plot_compare_images_plotly(images_or, imgs_filtered, cmap = cmap, high_contrast = high_contrast)
    
    def do_and_plot_percentile(channel_name, PERCENTILE, TH, cols, res, sample_images,sample_images_number, high_contrast):
        output.clear_output()
        plot_output_histogram.clear_output()
        plot_output_images.clear_output()  
        plot_output_comparison.clear_output()  
        plot_output_psnr.clear_output()  
        plot_output_ZOOM.clear_output() 

        files_channel = JN.defining_files(files,sample_images_number, sample_images)
        images_original, imgs_norm = JN.preprocess_images_pages(files_channel)
        
        imgs_channel = [images_original[i] for i in range(len(images_original))]
        norm_imgs_channel = [imgs_norm[i] for i in range(len(imgs_norm))]

        # select the channel 
        # channel_name = name of the channel . Map back to the index in the image . always by the same order in images? 
        CH = channel_names.index(channel_name)
        imgs_channel = [images_original[i][np.newaxis,...,CH] for i in range(len(images_original))] 
        norm_imgs_channel = [imgs_norm[i][np.newaxis,...,CH] for i in range(len(imgs_norm))] 

        
        
        norm_imgs_channel,imgs_filtered = JN.calculus_multitiff(imgs_ch=norm_imgs_channel, 
                                                                             PERCENTILE = PERCENTILE,
                                                                             TH = TH)

        with plot_output_histogram:
            set_params(res=50)
            JN.histogram_one_channel_all_img(norm_imgs_channel)
            JN.histogram_one_channel_all_img(imgs_filtered)

        with plot_output_images:
            plot_percentile(imgs_filtered, dropdown_cols.value, dropdown_res.value)

        with plot_output_comparison:            
            plot_comparison([obj[0] for obj in imgs_channel],[obj[0] for obj in imgs_filtered] , dropdown_res.value, 
                            high_contrast = dropdown_high_constrast_compare.value)

        with plot_output_ZOOM:
            plot_comparison_zoom([obj[0] for obj in imgs_channel],[obj[0] for obj in imgs_filtered], dropdown_res.value, 
                            high_contrast = dropdown_high_constrast_compare.value)


#         with plot_output_psnr:
#             plot_psnr(imgs_channel,norm_imgs_channel, imgs_filtered)
        return imgs_channel,norm_imgs_channel,imgs_filtered




    # dropdown functions 
    def dropdown_ch_name_eventhandler(change):
        imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(change.new,dropdown_percentile.value,dropdown_th.value,dropdown_cols.value, dropdown_res.value,dropdown_sample_images.value,
                        dropdown_sample_images_number.value,dropdown_high_constrast_compare.value)
    
    def dropdown_th_eventhandler(change):
        imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(dropdown_ch_name.value, dropdown_percentile.value, change.new,dropdown_cols.value, dropdown_res.value,dropdown_sample_images.value,
                        dropdown_sample_images_number.value,dropdown_high_constrast_compare.value)
    
    def dropdown_percentile_eventhandler(change):
        imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(dropdown_ch_name.value,change.new,dropdown_th.value,dropdown_cols.value, dropdown_res.value,dropdown_sample_images.value,
                        dropdown_sample_images_number.value,dropdown_high_constrast_compare.value)   

    def dropdown_cols_eventhandler(change):
        imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(
            dropdown_ch_name.value,dropdown_percentile.value,dropdown_th.value,
            change.new, dropdown_res.value,dropdown_sample_images.value,dropdown_sample_images_number.value,
        dropdown_high_constrast_compare.value)

    def dropdown_res_eventhandler(change):
            imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(
                dropdown_ch_name.value,dropdown_percentile.value,dropdown_th.value,
                dropdown_cols.value, change.new,dropdown_sample_images.value, dropdown_sample_images_number.value,
            dropdown_high_constrast_compare.value)

    def dropdown_sample_images_eventhandler(change):
            imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(
                dropdown_ch_name.value,dropdown_percentile.value,dropdown_th.value,
                dropdown_cols.value, dropdown_res.value,change.new,dropdown_sample_images_number.value,
            dropdown_high_constrast_compare.value)

    def dropdown_sample_images_number_eventhandler(change):
            imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(
                dropdown_ch_name.value,dropdown_percentile.value,dropdown_th.value,
                dropdown_cols.value, dropdown_res.value,dropdown_sample_images.value,change.new, dropdown_high_constrast_compare.value)
            
    def dropdown_high_constrast_compare_eventhandler(change):
            imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(
                dropdown_ch_name.value,dropdown_percentile.value,dropdown_th.value,
                dropdown_cols.value, dropdown_res.value,dropdown_sample_images.value,
                dropdown_sample_images_number.value,change.new)
            
            
    # create outputs
    output = widgets.Output()
    plot_output_histogram = widgets.Output()
    plot_output_images = widgets.Output()
    plot_output_comparison = widgets.Output()
    plot_output_psnr = widgets.Output()
    plot_output_ZOOM = widgets.Output()

    # clear outputs
    plot_output_histogram.clear_output()
    plot_output_images.clear_output()
    plot_output_comparison.clear_output()
    plot_output_psnr.clear_output()
    plot_output_ZOOM.clear_output()


    
    # define dropdown plots images
    dropdown_cols = widgets.Dropdown(options = [1,2,3,4,5], description='Columns to display:')
    dropdown_res = widgets.Dropdown(options = [50,100,200,300,400,500], description='Image resolution:')
    dropdown_sample_images = widgets.Dropdown(options = ['top', 'bottom', 'random', 'all'], 
                                              description='Display a subset of images:')
    dropdown_sample_images_number = widgets.Dropdown(options = [1,range(2,24,2)], description='Display a subset of images:')
    dropdown_high_constrast_compare = widgets.Dropdown(options = ['False', 'True'], description='Display on high constrast:')
    # define dropdown Denoise Parameters
    dropdown_th = widgets.Dropdown(options = [None,0.1,0.2,0.25,0.3,0.4,0.5,0.6,0.7,0.75,0.8,0.9, 
                                              'otsu', 'isodata', 'Li', 'Yen', 'triangle', 'mean', 'local'
                                             ], description='threshold:')
    dropdown_percentile = widgets.Dropdown(options = [None,10,20,25,30,40,50,60,70,75,80,90,5,95,'p50consecutive'], description='Percentile:')
    dropdown_ch_name = widgets.Dropdown(options = channel_names, description='Channel:')
    
    dropdown_ch_name.observe(dropdown_ch_name_eventhandler, names='value')
    dropdown_th.observe(dropdown_th_eventhandler, names='value')
    dropdown_percentile.observe(dropdown_percentile_eventhandler, names='value')

    dropdown_cols.observe(dropdown_cols_eventhandler, names='value')
    dropdown_res.observe(dropdown_res_eventhandler, names='value')
    dropdown_sample_images.observe(dropdown_sample_images_eventhandler, names='value')
    dropdown_sample_images_number.observe(dropdown_sample_images_number_eventhandler, names='value')
    dropdown_high_constrast_compare.observe(dropdown_high_constrast_compare_eventhandler, names='value')


    
    # Dashboard
    input_widgets = widgets.HBox([dropdown_ch_name, dropdown_percentile,dropdown_th])
    input_widgetsimages = widgets.HBox([dropdown_sample_images_number, dropdown_sample_images])
    input_widgetsimages2 = widgets.HBox([dropdown_res,dropdown_cols, dropdown_high_constrast_compare])

    # create a container for the output. Tab is great for this. The 1st tab will host the dataframe and the 2nd one the graph
    tab = widgets.Tab([plot_output_images, plot_output_histogram, plot_output_comparison,plot_output_ZOOM,  plot_output_psnr])
    tab.set_title(0, 'Images')
    tab.set_title(1, 'Histogram')
    tab.set_title(2, 'Compare images')
    tab.set_title(3, 'Compare ZOOM')
    tab.set_title(4, 'PSNR')

    # display(tab)
     # Define the layout for the widget container
    dashboard_layout = widgets.Layout(height='auto', overflow='auto',flex='1', max_height='100000px')
    dashboard = widgets.VBox([input_widgets, input_widgetsimages,input_widgetsimages2,tab], layout=dashboard_layout)
    display(dashboard)
    

In [4]:
from IPython.display import display, HTML
display(HTML("<style>div.output_scroll { height: 200em; }</style>"))
from IPython.display import clear_output
import ipywidgets as widgets

PATH = widgets.Text(value='path_to_data', description='Path:', disabled=False)
button = widgets.Button(description="Change Path")
output_widget = widgets.Output()

result=[]
def update_output(button):
    global directory_path, result, channel_names, images_names
    with output_widget:
        clear_output(wait=True)
        directory_path = PATH.value
        
        if directory_path:
            print("Path provided:", directory_path)
            # result = [y for x in os.walk(path) for y in glob(os.path.join(x[0], '*tif'))]
            images_names = [file for file in os.listdir(directory_path) if "tif" in file]
            result = [os.path.join(directory_path, file) for file in images_names]
            
            specs = FileSpecifics(result[0], multitiff=True)
            channel_names = specs.channel_names
#             if channel_names = None: 
#                 first_image, channel_names = IP.parse_image_pages_namesCH(result[0])
#                 channel_names = range(0,first_image.shape[2])
        
            widget(result, channel_names)

button.on_click(update_output)

display(PATH)
display(button)
display(output_widget)

Text(value='path_to_data', description='Path:')

Button(description='Change Path', style=ButtonStyle())

Output()

In [5]:
def save_imgs(files, images_names, path_for_results, binary_masks = False):
    if not os.path.exists(path_for_results):
        os.makedirs(path_for_results)

    #does all the files 
    images_original, imgs_norm = JN.preprocess_images_pages(files)
    imgs_channel = [images_original[i] for i in range(len(images_original))]
    norm_imgs_channel = [imgs_norm[i] for i in range(len(imgs_norm))]
    
    
    # grab the th_list and the percentile_list 
    download_table = edit_table
    th_list = list(download_table.value['threshold'])
    percentile_list = list(download_table.value['percentile'])
    
    csv_path = os.path.join(str(path_for_results), 'denoiseParameters.csv')
#     download_table.download(csv_path) # download in the wrong folder 

    df = pd.DataFrame(download_table.value)
    df.to_csv(csv_path, index=False)
    
    imgs_filtered = map(lambda i: JN.calculus_multitiff_lists(i,th_list, percentile_list), norm_imgs_channel)
    imgs_filtered = list(imgs_filtered)

    if binary_masks:
        imgs_filtered = [np.where(a > 0, 1, 0) for a in imgs_filtered]

    # get the names of images
    names_save = [str(path_for_results + sub) for sub in images_names if 'tif' in sub]
      
    if isinstance(channel_names[0], str):
        images_final = map(lambda p, f: IPrep.save_img_ch_names_pages(p, f, ch_last=True, channel_names=channel_names), imgs_filtered, names_save)

    else:
        # will not save channel names 
        images_final = map(lambda p, f: IPrep.save_images(p, f, ch_last=True), imgs_filtered, names_save)
    
    
    it = list(images_final)
    print('Images saved')

# events for saving     
def btn_eventhandler(obj):
    print('Hello from the {} button!'.format(obj.description))
    print('Your images are being saved -np32 format ')

    path_for_results = txtbox.value
    save_imgs(files=result, images_names=images_names, path_for_results=path_for_results)

def btn_bin_eventhandler(obj):
    print('Hello from the {} button!'.format(obj.description))
    print('Your Binary images are being saved - np32 format')
    path_for_results = txtbox.value
    save_imgs(files=result,images_names=images_names,path_for_results=path_for_results, binary_masks =True)      

def txtbox_eventhandler(obj):
    path_for_results = obj.description

# Function to generate the table and other saving widgets
def generate_saving_widgets():
    global edit_table, txtbox, btn_save, btn_save_bin
    clear_output(wait=True)  # Clear the output before displaying the saving options
    sel_df = pd.DataFrame({'threshold': [0.1]*len(channel_names), 'percentile': [50]*len(channel_names)}, index=channel_names)
    tabulator_editors = {'threshold': {'type': 'number', 'max': 1, 'step': 0.1}, 'percentile': {'type': 'number', 'max': 100, 'step': 5}}
    edit_table = pn.widgets.Tabulator(sel_df, editors=tabulator_editors, configuration={'columnDefaults': {'headerSort': False}})
    txtbox = widgets.Text(value='ResultsPercentile/', placeholder='Type something', description='PathSave:', disabled=False)
    btn_save = widgets.Button(description='Save')
    btn_save_bin = widgets.Button(description='Save Binary')
    btn_save.on_click(btn_eventhandler)  # Assign event handler for Save button
    btn_save_bin.on_click(btn_bin_eventhandler)
    display(edit_table)
    display(txtbox)
    display(widgets.HBox([btn_save, btn_save_bin]))
    widgets_created = True

# Function to handle the click event of the "Saving Options" button
def saving_options_eventhandler(obj):
    generate_saving_widgets()

# "Saving Options" button
btn_saving_options = widgets.Button(description='Saving Options')
btn_saving_options.on_click(saving_options_eventhandler)
display(btn_saving_options)

Button(description='Saving Options', style=ButtonStyle())